In [ ]:
import warnings
warnings.filterwarnings('ignore')
# from datetime import datetime, date
import numpy as np
import pandas as pd
# import yfinance as yf

def pct_to_float(number):
    """Convert string to float, remove % char and set decimal point to '.'."""
    return float(number.strip("%").replace(".", "").replace(",", "."))

def LPA_VPA(ticker:str) :
    url=f'https://www.fundamentus.com.br/detalhes.php?papel={ticker}'
    r = requests.get(url, headers=header)
    PA = pd.read_html(r.text, decimal=',', thousands='.',encoding='ISO-8859-1' )[2]
    # print(PA[[4,5]].iloc[1:3])
    return PA[5].iloc[1], PA[5].iloc[2]


In [87]:
# dados fundamentus
import requests
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}
url = 'https://www.fundamentus.com.br/resultado.php'
#junta com a requests
r = requests.get(url, headers=header)
# read_html do pandas põe a tabela num dataframe
funda = pd.read_html(r.text, 
                     decimal=',', thousands='.',encoding='ISO-8859-1', 
                     converters={'ROE': pct_to_float,
                                 'ROIC': pct_to_float,
                                 'Div.Yield':pct_to_float,
                                 'Mrg Ebit':pct_to_float,
                                 'Mrg. Líq.':pct_to_float,
                                 'Cresc. Rec.5a':pct_to_float,
                                 },
)[0]
dfunds = pd.DataFrame(funda)

In [102]:
lpa = []
vpa=[]
for t in dfunds['Papel']:
    LPA,VPA= LPA_VPA(t)
    lpa.append(LPA)
    vpa.append(VPA)
dfunds['LPA'] = lpa
dfunds['VPA'] = vpa

In [106]:
dfunds.columns

Index(['Papel', 'Cotação', 'P/L', 'P/VP', 'PSR', 'Div.Yield', 'P/Ativo',
       'P/Cap.Giro', 'P/EBIT', 'P/Ativ Circ.Liq', 'EV/EBIT', 'EV/EBITDA',
       'Mrg Ebit', 'Mrg. Líq.', 'Liq. Corr.', 'ROIC', 'ROE', 'Liq.2meses',
       'Patrim. Líq', 'Dív.Brut/ Patrim.', 'Cresc. Rec.5a', 'LPA', 'VPA'],
      dtype='object')

In [41]:
# A empresa e Indicadores fundamentalistas dados do statusinvest.com.br
#(pode ter atraso, dados não são pegos diretamente no SI)

url = 'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/SI_Acoes.csv'
pd.set_option('display.max_columns', None)
fundsSI = pd.read_csv(url,sep=';' , decimal=',' ,thousands ='.' )
fundsSI.fillna(0)
fundsSI

,TICKER,PRECO,DY,P/L,P/VP,P/ATIVOS,MARGEM BRUTA,MARGEM EBIT,MARG. LIQUIDA,P/EBIT,EV/EBIT,DIVIDA LIQUIDA / EBIT,DIV. LIQ. / PATRI.,PSR,P/CAP. GIRO,P. AT CIR. LIQ.,LIQ. CORRENTE,ROE,ROA,ROIC,PATRIMONIO / ATIVOS,PASSIVOS / ATIVOS,GIRO ATIVOS,CAGR RECEITAS 5 ANOS,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA,VPA,LPA,PEG Ratio,VALOR DE MERCADO
0,AALR3,8.66,NaN,-3.89,1.01,0.37,30.10,1.60,-22.81,55.37,96.28,40.90,0.75,0.89,-4.58,-0.48,0.74,-25.92,-9.58,-0.51,0.37,0.62,0.42,0.13,NaN,8.598338e+05,8.58,-2.22,-0.04,1.024416e+09
1,ABCB4,24.75,6.37,6.05,1.02,0.10,25.50,17.06,15.50,5.50,5.50,NaN,NaN,0.94,-2.86,-0.10,0.05,16.95,1.73,NaN,0.10,0.91,0.11,18.70,17.80,1.662386e+07,24.15,4.09,-0.33,5.864168e+09
2,ABEV3,12.70,5.75,13.27,2.22,1.45,50.30,22.59,18.29,10.74,9.99,-0.75,-0.15,2.43,53.97,-1.99,1.11,16.71,10.94,18.63,0.65,0.34,0.60,10.72,15.72,2.518599e+08,5.73,0.96,0.88,2.000737e+11
3,ADHM3,1.56,NaN,-4.09,-1.29,105.12,100.00,-13548.48,-18860.61,-5.69,-5.69,0.00,NaN,770.90,-1.43,-116.16,0.00,-31.47,-2571.90,22.61,-81.72,82.72,0.14,NaN,NaN,NaN,-1.21,-0.38,-0.58,2.543977e+07
4,AERI3,0.76,NaN,-8.63,1.16,0.25,12.62,9.03,-3.39,3.24,6.75,3.51,1.26,0.29,1.15,-0.76,1.46,-13.42,-2.86,10.53,0.21,0.79,0.84,NaN,NaN,4.019594e+06,0.66,-0.09,-0.04,9.442270e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,WIZC3,6.26,7.11,10.29,2.18,0.44,58.84,37.07,9.26,2.57,2.84,0.26,0.22,0.95,-17.18,-0.56,0.90,21.18,4.24,22.73,0.20,0.59,0.46,12.37,-9.41,2.804296e+06,2.87,0.61,-0.27,1.001020e+09
616,WLMM3,24.00,1.66,8.27,1.30,1.06,12.97,7.04,4.99,5.87,5.85,-0.61,-0.14,0.41,2.42,-2.41,4.54,15.77,12.80,15.79,0.81,0.19,2.56,32.16,NaN,1.634088e+04,18.40,2.90,-1.21,9.620570e+08
617,WLMM4,28.44,1.54,9.80,1.55,1.25,12.97,7.04,4.99,6.95,5.85,-0.61,-0.14,0.49,2.87,-2.86,4.54,15.77,12.80,15.79,0.81,0.19,2.56,32.16,NaN,3.790545e+04,18.40,2.90,-1.43,9.620570e+08
618,YDUQ3,19.05,1.44,31.31,1.82,0.60,59.29,17.03,3.75,6.89,11.75,4.87,1.29,1.17,6.76,-0.81,1.52,5.81,1.92,9.55,0.33,0.67,0.51,6.20,-15.03,7.108171e+07,10.48,0.61,-0.06,5.888143e+09


In [42]:
fundsSI[fundsSI['TICKER'].str.contains('GOAU')]

,TICKER,PRECO,DY,P/L,P/VP,P/ATIVOS,MARGEM BRUTA,MARGEM EBIT,MARG. LIQUIDA,P/EBIT,EV/EBIT,DIVIDA LIQUIDA / EBIT,DIV. LIQ. / PATRI.,PSR,P/CAP. GIRO,P. AT CIR. LIQ.,LIQ. CORRENTE,ROE,ROA,ROIC,PATRIMONIO / ATIVOS,PASSIVOS / ATIVOS,GIRO ATIVOS,CAGR RECEITAS 5 ANOS,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA,VPA,LPA,PEG Ratio,VALOR DE MERCADO
294,GOAU3,10.33,15.39,3.89,0.59,0.14,17.5,16.6,3.8,0.89,1.25,0.38,0.25,0.15,0.50,-0.23,3.02,15.29,3.54,15.06,0.23,0.33,0.93,17.42,NaN,1051742.25,17.38,2.66,-0.09,1.044754e+10
295,GOAU4,10.00,15.90,3.76,0.58,0.13,17.5,16.6,3.8,0.86,1.25,0.38,0.25,0.14,0.48,-0.23,3.02,15.29,3.54,15.06,0.23,0.33,0.93,17.42,NaN,83356966.78,17.38,2.66,-0.09,1.044754e+10


In [43]:
fundsSI.columns

Index(['TICKER', 'PRECO', 'DY', 'P/L', 'P/VP', 'P/ATIVOS', 'MARGEM BRUTA',
       'MARGEM EBIT', 'MARG. LIQUIDA', 'P/EBIT', 'EV/EBIT',
       'DIVIDA LIQUIDA / EBIT', 'DIV. LIQ. / PATRI.', 'PSR', 'P/CAP. GIRO',
       'P. AT CIR. LIQ.', 'LIQ. CORRENTE', 'ROE', 'ROA', 'ROIC',
       'PATRIMONIO / ATIVOS', 'PASSIVOS / ATIVOS', 'GIRO ATIVOS',
       'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS', ' LIQUIDEZ MEDIA DIARIA',
       ' VPA', ' LPA', ' PEG Ratio', ' VALOR DE MERCADO'],
      dtype='object')

In [44]:
fundsSI['valor_intrinseco'] = np.sqrt(22.5 * fundsSI[' LPA'] * fundsSI[' VPA'])
fundsSI['Delta'] = fundsSI['valor_intrinseco'] - fundsSI['PRECO']
fundsSI.dropna(inplace=True)


In [45]:
fundsSI["Rank"]   = fundsSI['Delta'].rank(ascending=True, method="min")


In [46]:
fundsSI.sort_values(by="Rank", ascending=False).head(130)#, inplace=True)


,TICKER,PRECO,DY,P/L,P/VP,P/ATIVOS,MARGEM BRUTA,MARGEM EBIT,MARG. LIQUIDA,P/EBIT,EV/EBIT,DIVIDA LIQUIDA / EBIT,DIV. LIQ. / PATRI.,PSR,P/CAP. GIRO,P. AT CIR. LIQ.,LIQ. CORRENTE,ROE,ROA,ROIC,PATRIMONIO / ATIVOS,PASSIVOS / ATIVOS,GIRO ATIVOS,CAGR RECEITAS 5 ANOS,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA,VPA,LPA,PEG Ratio,VALOR DE MERCADO,valor_intrinseco,Delta,Rank
75,BMKS3,360.00,16.05,7.51,0.80,0.70,28.02,25.71,136.17,39.75,1.56,-38.20,-0.77,10.22,0.95,-2.85,36.30,10.66,9.28,-1.63,0.87,0.13,0.07,2.51,22.12,7.594770e+03,450.03,47.95,-1.78,1.637100e+08,696.797041,336.797041,167.0
156,CLSC3,65.02,2.59,4.39,0.79,0.22,15.15,9.04,5.50,2.67,4.59,1.90,0.56,0.24,-3.47,-0.31,0.83,17.93,4.93,13.59,0.27,0.73,0.90,7.34,53.76,2.855413e+04,82.64,14.82,0.06,2.533965e+09,166.000928,100.980928,166.0
157,CLSC4,66.15,2.80,4.47,0.80,0.22,15.15,9.04,5.50,2.71,4.59,1.90,0.56,0.25,-3.53,-0.31,0.83,17.93,4.93,13.59,0.27,0.73,0.90,7.34,53.76,2.692496e+05,82.64,14.82,0.06,2.533965e+09,166.000928,99.850928,165.0
404,MSPA4,40.00,0.15,7.79,0.31,0.18,38.88,23.14,17.10,5.75,8.52,1.98,0.11,1.33,5.25,-0.20,1.56,3.94,2.30,4.10,0.58,0.42,0.13,6.46,-2.91,6.023000e+03,130.22,5.14,-0.04,2.913945e+08,122.718959,82.718959,164.0
410,MTSA4,47.98,2.64,4.84,0.98,0.75,29.63,13.88,13.56,4.72,4.75,-1.40,-0.29,0.66,1.32,-3.39,3.77,20.20,15.55,15.32,0.77,0.23,1.15,27.16,46.00,1.296467e+05,49.09,9.92,0.81,5.498764e+08,104.675155,56.695155,163.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,CSRN3,28.40,3.91,8.19,3.34,0.83,31.55,26.27,16.02,4.99,7.58,2.59,1.73,1.31,-144.65,-1.11,0.98,40.77,10.10,18.81,0.25,0.75,0.63,10.30,25.36,7.593290e+03,8.51,3.47,2.09,4.762954e+09,25.776312,-2.623688,42.0
597,VBBR3,24.18,1.77,13.84,2.08,0.69,4.58,2.47,1.24,6.94,9.29,2.36,0.71,0.17,2.23,-1.51,2.31,15.01,4.97,11.75,0.33,0.67,4.01,16.50,12.07,1.778232e+08,11.64,1.75,7.08,2.816970e+10,21.408526,-2.771474,41.0
329,INTB3,19.68,1.06,12.38,2.61,1.37,31.46,11.78,12.53,13.16,12.62,-0.54,-0.11,1.55,3.00,-5.00,2.68,21.10,11.04,13.25,0.52,0.47,0.88,23.09,23.17,2.455510e+07,7.53,1.59,0.51,6.447387e+09,16.412975,-3.267025,40.0
275,FLRY3,15.80,2.75,23.15,1.67,0.75,27.02,14.26,6.35,10.31,14.01,3.70,0.60,1.47,9.39,-0.99,1.48,7.21,3.24,7.99,0.45,0.55,0.51,13.37,3.10,4.126425e+07,9.46,0.68,3.14,8.645618e+09,12.030711,-3.769289,39.0


In [47]:
fundsSI[fundsSI['TICKER'].str.contains('GOAU')]

,TICKER,PRECO,DY,P/L,P/VP,P/ATIVOS,MARGEM BRUTA,MARGEM EBIT,MARG. LIQUIDA,P/EBIT,EV/EBIT,DIVIDA LIQUIDA / EBIT,DIV. LIQ. / PATRI.,PSR,P/CAP. GIRO,P. AT CIR. LIQ.,LIQ. CORRENTE,ROE,ROA,ROIC,PATRIMONIO / ATIVOS,PASSIVOS / ATIVOS,GIRO ATIVOS,CAGR RECEITAS 5 ANOS,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA,VPA,LPA,PEG Ratio,VALOR DE MERCADO,valor_intrinseco,Delta,Rank


### Porque 22,5?

O número 22,5 na Fórmula de Graham é um fator de ponderação que tem um propósito específico. Essa constante foi escolhida por Graham pra ajustar a avaliação do preço justo de uma ação com base na taxa de crescimento anual esperada da empresa.

O número 22,5 é, na verdade, o resultado da multiplicação de 8,5 por 2,65 (8,5 x 2,65 = 22,5). O número 8,5 é a base que Graham considerou razoável pra uma empresa com taxa de crescimento zero, ou seja, uma empresa que não cresce.

Já o número 2,65 representa a média do retorno exigido pelos investidores no mercado de ações durante a época de Graham, que era de aproximadamente 4,4% acima da taxa de retorno dos títulos do Tesouro dos Estados Unidos.

O fator 22,5 ajuda a ajustar o preço justo com base no crescimento da empresa e na expectativa de retorno dos investidores.

Esse ajuste garante que a Fórmula de Graham considere a taxa de crescimento anual esperada e reflita uma avaliação mais realista do preço justo de uma ação.

No próximo tópico, abordaremos a calculadora do preço justo usando a Fórmula de Graham.